In [ ]:
# Jax imports
import jax
import jax.numpy as jnp
import numpy as np
from jax import config

config.update("jax_enable_x64", False)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# IO utils
from jax_2dtm.io import load_grid_as_cloud

In [ ]:
# Image simulator
from jax_2dtm.simulator import NufftScattering, GaussianScattering, Cloud
from jax_2dtm.simulator import ScatteringImage, OpticsImage, GaussianImage
from jax_2dtm.simulator import AntiAliasingFilter, WhiteningFilter
from jax_2dtm.simulator import EulerPose, CTFOptics, Intensity, WhiteNoise, ExponentialNoise, ParameterState
from jax_2dtm.utils import fft, ifft, fftfreqs, pad, powerspectrum, radial_average

In [ ]:
# Volume filename and metadata
pixel_size = 5.28  # Angstroms
filename = "../tests/data/3jar_13pf_bfm1_ps5_28.mrc"

In [ ]:
# Configure image data and read template as point cloud
config = NufftScattering((95, 95), pixel_size, pad_scale=1.4, eps=1e-6)
#config = GaussianScattering((131, 131), pixel_size, pad=2, scale=1/3)
cloud = load_grid_as_cloud(filename, config)

In [ ]:
# Initialize model, parameters, and compute image
pose = EulerPose(-50.0, 50.0, np.pi / 4, np.pi / 10, np.pi / 4)
optics = CTFOptics()
noise = ExponentialNoise(key=jax.random.PRNGKey(seed=0), kappa=0.3, xi=4.0, alpha=1.0)
intensity = Intensity()
state = ParameterState(pose=pose, optics=optics, noise=noise, intensity=intensity)

In [ ]:
scattering_model = ScatteringImage(config=config, cloud=cloud, state=state)
optics_model = OpticsImage(config=config, cloud=cloud, state=state)
noisy_model = GaussianImage(config=config, cloud=cloud, state=state)

In [ ]:
# Plot models
fig, axes = plt.subplots(ncols=3, figsize=(12, 6))
ax1, ax2, ax3 = axes
ax1.imshow(ifft(scattering_model()), origin="lower", cmap="gray")
ax2.imshow(ifft(optics_model()), origin="lower", cmap="gray")
ax3.imshow(ifft(noisy_model()), origin="lower", cmap="gray")

Now, let's test altering the image filters.

In [ ]:
# Instantiate image filters
fig, ax = plt.subplots(figsize=(4, 4))
micrograph_freqs = fftfreqs((1200, 800), pixel_size)
micrograph = noise.sample(config.pixel_size * micrograph_freqs)
whiten = WhiteningFilter(config.pixel_size * config.freqs, config.pixel_size * micrograph_freqs, micrograph)
plt.imshow(ifft(whiten(noisy_model())), origin="lower", cmap="gray")

We see that the whitening filter looks right, empirically! Now generate an image at a colored noise model with a given whitening filter

In [ ]:
# Show forward model for a whitened image
fig, ax = plt.subplots(figsize=(4, 4))
filters = [AntiAliasingFilter(config.pixel_size * config.padded_freqs), WhiteningFilter(config.pixel_size * config.padded_freqs, config.pixel_size * micrograph_freqs, micrograph)]
filtered_model = GaussianImage(config=config, cloud=cloud, state=state, filters=filters)
ax.imshow(ifft(filtered_model.render(state)), origin="lower", cmap="gray")

In [ ]:
# Visualize filters
fig, axes = plt.subplots(ncols=2, figsize=(8, 6))
ax1, ax2 = axes
antialias, whiten = filters
im1 = ax1.imshow(whiten.filter, cmap="gray")
im2 = ax2.imshow(antialias.filter, cmap="gray")

Computing an image is straight-forward, but really we want to define a function that can be arbitrarily transformed by JAX and evaulated at subsets of the parameters.

In [ ]:
# Define subset of parameters over which to evaluate model, and jitted model
params = dict(view_phi=np.pi, defocus_u=9000.0, sigma=1.0, N=1.0, mu=10.0)
jitted_noisy_model = jax.jit(lambda params: noisy_model(params))

In [ ]:
# Benchmark jitted pipeline
jitted_noisy_image = jitted_noisy_model(params)

In [ ]:
# Benchmark non-jitted pipeline
noisy_image = noisy_model(params)

Now, we can also use the model to compute the likelihood. Let's evaulate the likelihood at the simulated data, and visualize the residuals.

In [ ]:
# Initialize the model and plot residuals
fig, axes = plt.subplots(ncols=3, figsize=(12, 6))
ax1, ax2, ax3 = axes
observed = noisy_model()
model = GaussianImage(config=config, cloud=cloud, state=state, observed=observed)
simulated, observed, residuals = model.render(), model.observed, model.residuals()
ax1.imshow(ifft(simulated), origin="lower", cmap="gray")
ax2.imshow(ifft(observed), origin="lower", cmap="gray")
ax3.imshow(ifft(residuals), origin="lower", cmap="gray")


In [ ]:
# Compute likelihood
loss = jax.jit(lambda params: model(params))
grad_loss = jax.jit(jax.grad(lambda params: model(params)))

In [ ]:
# Benchmark jitted pipeline
likelihood = loss(params)
likelihood

In [ ]:
# Benchmark gradient
grad = grad_loss(params)
grad